# URL Embedding Clustering

In [1]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append(os.path.abspath(".."))

from hdbscan import HDBSCAN
from sklearn import metrics
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from url2vec.util.plotter import *
from url2vec.util.metrics import *
from url2vec.util.seqmanager import *

import plotly.plotly as py
from plotly.graph_objs import *
from __future__ import print_function
from plotly.tools import FigureFactory as FF

In [2]:
# available datasets
# cs.illinois.edu    cs.stanford.edu    eecs.mit.edu    cs.princeton.edu    cs.ox.ac.uk
site = "cs.princeton.edu"
words = "10000"
depth = "7"

The crawling proccess has been done in two different ways:

- **No costraint**: the crawler follows a random outlink from all of the outlinks in a given page
- **List costraint**: the crawler follows a random outlink but only from the outlinks in "lists"

## Word2Vec - no-costraint
Word embedding algorithm. Word2vec is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. 

Here we're loading all the files that the crawler has generated to train word2vec model.

See the [Dataset README](https://github.com/chrisPiemonte/url2vec/tree/master/dataset "Dataset") for further information.

In [3]:
nocostraint_path = os.getcwd() + "/../dataset/" + site + "/no_constraint/words" + words + "_depth" + depth + "/"
nocostraint_urlmap_path = nocostraint_path + "urlsMap.txt"
nocostraint_seq_path = nocostraint_path + "sequenceIDs.txt"

nocostraint_urlmap = get_urlmap(nocostraint_urlmap_path)

Training the model. No need to keep everything in RAM so we're passing two generators.

**PARAMETERS**:

- **min_count**: ignore all words with total frequency lower than this
- **window**: is the maximum distance between the current and predicted word within a sentence
- **negative**: if > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). Default is 5. If set to 0, no negative samping is used

In [4]:
# because of generator
vocab_sequences_nc = get_sequences(nocostraint_seq_path)
train_sequences_nc = get_sequences(nocostraint_seq_path)

w2v_model_nc = Word2Vec(min_count=1, window=5, negative=5, sg=1)
%time w2v_model_nc.build_vocab(vocab_sequences_nc)
%time w2v_model_nc.train(train_sequences_nc)

CPU times: user 580 ms, sys: 4 ms, total: 584 ms
Wall time: 581 ms
CPU times: user 4.02 s, sys: 32 ms, total: 4.05 s
Wall time: 1.3 s


601750

In [5]:
url_not_in_sequences = list(set(nocostraint_urlmap) - set(w2v_model_nc.vocab))
print(len(url_not_in_sequences))
for url in url_not_in_sequences:
    del nocostraint_urlmap[url]

3843


### t-SNE
Applying t-SNE for dimensionality reduction. We need two dimensional vectors for visualization purposes.

In [6]:
print(len(nocostraint_urlmap))
print(len(w2v_model_nc.vocab))
# 100-dim vecs
wordvecs_nc = np.array([w2v_model_nc[key] for key in nocostraint_urlmap], dtype="float64")

# URL labels
urls_nc = [nocostraint_urlmap[key] for key in nocostraint_urlmap]

# 2-dim vecs
tsne = TSNE(n_components=2)
twodim_wordvecs_nc = %time tsne.fit_transform(wordvecs_nc)

10909
10909
CPU times: user 4min 26s, sys: 17 s, total: 4min 43s
Wall time: 3min 4s


## Clustering - No-Costraint Word2Vec model

### DBSCAN
DBSCAN - Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density.

**PARAMETERS**:

- **eps** : The maximum distance between two samples for them to be considered as in the same neighborhood.
- **min_samples** : The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

In [7]:
dbscan = DBSCAN(eps=0.8, min_samples=7)
%time dbscan.fit(wordvecs_nc)
dbscan_labels_nc = dbscan.labels_

dbscan_colors_nc = [get_color(clust) for clust in dbscan_labels_nc]

print("Clusters found with DBSCAN:", len(set(dbscan_labels_nc)))
print ([label for label in set(dbscan_labels_nc)])
print("\n\n")

CPU times: user 17.7 s, sys: 160 ms, total: 17.9 s
Wall time: 17.6 s
Clusters found with DBSCAN: 16
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, -1]





### DBSCAN Plot

In [8]:
dbscan_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, dbscan_colors_nc)
py.iplot(dbscan_data_nc, filename='Word Vectors Nocostraint - Scatter plot DBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/0" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_DBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:0"  src="https://plot.ly/embed.js" async></script>
</div>

### HDBSCAN
HDBSCAN - Hierarchical Density-Based Spatial Clustering of Applications with Noise. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

**PARAMETERS**:
- **min_cluster_size** : minimum nodes to form a cluster

In [9]:
hdbscan = HDBSCAN(min_cluster_size=10)
%time hdbscan_labels_nc = hdbscan.fit_predict(wordvecs_nc)

hdbscan_colors_nc = [get_color(clust) for clust in hdbscan_labels_nc]

print("Clusters found with HDBSCAN:", len(set(hdbscan_labels_nc)))
print([label for label in set(hdbscan_labels_nc)])
print("\n\n")

CPU times: user 14.3 s, sys: 4 ms, total: 14.3 s
Wall time: 14.3 s
Clusters found with HDBSCAN: 27
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, -1]





### HDBSCAN Plot

In [13]:
hdbscan_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, hdbscan_colors_nc)
py.iplot(hdbscan_data_nc, filename='Word Vectors Nocostraint - Scatter plot HDBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/2" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_HDBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:2"  src="https://plot.ly/embed.js" async></script>
</div>

### K-MEANS
The first step is to initialize the algorithm by choosing K initial cluster centroid locations. This is typically done by randomly choosing K points from the input set. With these initial K centroids, the algorithm proceeds by repeating the following two main steps:

- Cluster Assignment - Here, each observation (e.g., each point in the data set) is assigned to a cluster centroid such that the WCSS objective function is minimized. This can often be translated to assigning each observation to the closest cluster centroid (which coincidentally minimizes WCSS for many distance metrics), though for some distance metrics and spaces this need not be the case.
- Update Centroids - After all of the input observations have been assigned to a cluster centroid, each centroid is re-computed. For each cluster, the new centroid is computed by averaging the observations that were assigned to it (e.g., computing the 'mean' of the observations).

These steps are repeated until the algorithm "converges".

**PARAMETERS**:

- **n_clusters**: number of clusters

In [14]:
kmeans = KMeans(n_clusters=30)
%time kmeans.fit(wordvecs_nc)

kmeans_labels_nc = kmeans.labels_

kmeans_colors_nc = [get_color(clust) for clust in kmeans_labels_nc]

print("Clusters found with K-MEANS:", len(set(kmeans_labels_nc)))
print([label for label in set(kmeans_labels_nc)])

CPU times: user 10.8 s, sys: 204 ms, total: 11 s
Wall time: 2.79 s
Clusters found with K-MEANS: 30
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


### K-MEANS Plot

In [15]:
kmeans_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, kmeans_colors_nc)
py.iplot(kmeans_data_nc, filename='Word Vectors Nocostraint - Scatter plot K-MEANS')

<div>
    <a href="https://plot.ly/~chrispolo/4" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:4"  src="https://plot.ly/embed.js" async></script>
</div>

### GROUND TRUTH

In [16]:
gt = GroundTruth(os.getcwd() + "/../dataset/" + site + "/ground_truth/urlToMembership.txt")
ground_truth_nc = [int(gt.get_groundtruth(nocostraint_urlmap[key])) for key in nocostraint_urlmap]

real_colors_nc = [get_color(n) for n in ground_truth_nc]

print("Clusters found manually:", len(set(ground_truth_nc)))
print([label for label in set(ground_truth_nc)])

Url not found
Url not found
Clusters found manually: 33
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, -1]


### GROUND TRUTH Plot

In [17]:
groundtruth_data_nc = scatter_plot(twodim_wordvecs_nc, urls_nc, real_colors_nc)
py.iplot(groundtruth_data_nc, filename='Word Vectors Nocostraint - Scatter plot Ground Truth')

<div>
    <a href="https://plot.ly/~chrispolo/56" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/nc_wordvectors_scatter_plot_GROUNDTRUTH.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:56"  src="https://plot.ly/embed.js" async></script>
</div>

---

## Word2Vec - list-costraint
Word embedding alorithm. Word2vec is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. 

Here we're loading all the files that the crawler has generated to train word2vec model.

See the [Dataset README](https://github.com/chrisPiemonte/url2vec/tree/master/dataset "Dataset") for further information.

In [18]:
listcostraint_path        = os.getcwd() + "/../dataset/" + site + "/list_constraint/words" + words + "_depth" + depth + "/"
listcostraint_urlmap_path = listcostraint_path + "urlsMap.txt"
listcostraint_seq_path    = listcostraint_path + "sequenceIDs.txt"

Training the model. No need to keep everything in RAM so we're passing two generators.

**PARAMETERS**:

- **min_count**: ignore all words with total frequency lower than this
- **window**: is the maximum distance between the current and predicted word within a sentence
- **negative**: if > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). Default is 5. If set to 0, no negative samping is used

In [19]:
# because of generator
vocab_seq_lc = get_sequences(listcostraint_seq_path)
train_seq_lc = get_sequences(listcostraint_seq_path)

word2vec_lc = Word2Vec(min_count=1, window=5, negative=5, sg=1)
%time word2vec_lc.build_vocab(vocab_seq_lc)
%time word2vec_lc.train(train_seq_lc)

CPU times: user 960 ms, sys: 12 ms, total: 972 ms
Wall time: 968 ms
CPU times: user 2.94 s, sys: 60 ms, total: 3 s
Wall time: 939 ms


486299

### t-SNE
Applying t-SNE for dimensionality reduction. We need two dimensional vectors for visualization purposes.

In [20]:
listcostraint_urlmap = get_urlmap(listcostraint_urlmap_path)

url_not_in_sequences = list(set(listcostraint_urlmap) - set(word2vec_lc.vocab))
print(len(url_not_in_sequences))
for url in url_not_in_sequences:
    del listcostraint_urlmap[url]

# 100-dim vecs
wordvecs_lc = np.array([word2vec_lc[key] for key in listcostraint_urlmap], dtype="float64")

# URL labels
urls_lc = [listcostraint_urlmap[key] for key in listcostraint_urlmap]

# 2-dim vecs
tsne = TSNE(n_components=2)
twodim_wordvecs_lc = %time tsne.fit_transform(wordvecs_lc)

16
CPU times: user 30.1 s, sys: 776 ms, total: 30.9 s
Wall time: 30.2 s


### DBSCAN
DBSCAN - Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density.

**PARAMETERS**:

- **eps** : The maximum distance between two samples for them to be considered as in the same neighborhood.
- **min_samples** : The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

In [21]:
dbscan = DBSCAN(eps=0.7, min_samples=5)
%time dbscan.fit(wordvecs_lc)
dbscan_labels_lc = dbscan.labels_

dbscan_colors_lc = [get_color(clust) for clust in dbscan_labels_lc]

print("Clusters found with DBSCAN:", len(set(dbscan_labels_lc)))
print ([label for label in set(dbscan_labels_lc)])
print("\n\n")

CPU times: user 3.74 s, sys: 36 ms, total: 3.78 s
Wall time: 3.72 s
Clusters found with DBSCAN: 11
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1]





### DBSCAN Plot

In [22]:
dbscan_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, dbscan_colors_lc)
py.iplot(dbscan_data_lc, filename='Word Vectors Lists - DBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/60" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_DBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:60"  src="https://plot.ly/embed.js" async></script>
</div>

### HDBSCAN
HDBSCAN - Hierarchical Density-Based Spatial Clustering of Applications with Noise. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

**PARAMETERS**:
- **min_cluster_size** : minimum nodes to form a cluster

In [23]:
hdbscan = HDBSCAN(min_cluster_size=8)
%time hdbscan_labels_lc = hdbscan.fit_predict(wordvecs_lc)

hdbscan_colors_lc = [get_color(clust) for clust in hdbscan_labels_lc]

print("Clusters found with HDBSCAN:", len(set(hdbscan_labels_lc)))
print([label for label in set(hdbscan_labels_lc)])
print("\n\n")

CPU times: user 1.41 s, sys: 0 ns, total: 1.41 s
Wall time: 1.41 s
Clusters found with HDBSCAN: 17
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, -1]





### HDBSCAN Plot

In [24]:
hdbscan_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, hdbscan_colors_lc)
py.iplot(hdbscan_data_lc, filename='Word Vectors Lists - Scatter plot HDBSCAN')

<div>
    <a href="https://plot.ly/~chrispolo/62" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_HDBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:62"  src="https://plot.ly/embed.js" async></script>
</div>

### K-MEANS
The first step is to initialize the algorithm by choosing K initial cluster centroid locations. This is typically done by randomly choosing K points from the input set. With these initial K centroids, the algorithm proceeds by repeating the following two main steps:

- Cluster Assignment - Here, each observation (e.g., each point in the data set) is assigned to a cluster centroid such that the WCSS objective function is minimized. This can often be translated to assigning each observation to the closest cluster centroid (which coincidentally minimizes WCSS for many distance metrics), though for some distance metrics and spaces this need not be the case.
- Update Centroids - After all of the input observations have been assigned to a cluster centroid, each centroid is re-computed. For each cluster, the new centroid is computed by averaging the observations that were assigned to it (e.g., computing the 'mean' of the observations).

These steps are repeated until the algorithm "converges".

**PARAMETERS**:

- **n_clusters**: number of clusters

In [25]:
kmeans = KMeans(n_clusters=30)
%time kmeans.fit(wordvecs_lc)

kmeans_labels_lc = kmeans.labels_

kmeans_colors_lc = [get_color(clust) for clust in kmeans_labels_lc]

print("Clusters found with K-MEANS:", len(set(kmeans_labels_lc)))
print([label for label in set(kmeans_labels_lc)])

CPU times: user 7.25 s, sys: 176 ms, total: 7.42 s
Wall time: 1.87 s
Clusters found with K-MEANS: 30
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


### K-MEANS Plot

In [26]:
kmeans_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, kmeans_colors_lc)
py.iplot(kmeans_data_lc, filename='Word Vectors Lists - Scatter plot K-MEANS')

<div>
    <a href="https://plot.ly/~chrispolo/64" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:64"  src="https://plot.ly/embed.js" async></script>
</div>

### GROUND TRUTH

In [27]:
gt = GroundTruth(os.getcwd() + "/../dataset/" + site + "/ground_truth/urlToMembership.txt")
ground_truth_lc = [int(gt.get_groundtruth(listcostraint_urlmap[key])) for key in listcostraint_urlmap]

real_colors_lc = [get_color(n) for n in ground_truth_lc]

print("Clusters found manually:", len(set(ground_truth_lc)))
print([label for label in set(ground_truth_lc)])

Clusters found manually: 25
[0, 1, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, -1]


### GROUND TRUTH Plot

In [28]:
groundtruth_data_lc = scatter_plot(twodim_wordvecs_lc, urls_lc, real_colors_lc)
py.iplot(groundtruth_data_lc, filename='Word Vectors Lists - Scatter plot Ground Truth')

<div>
    <a href="https://plot.ly/~chrispolo/66" 
        target="_blank" title="y" 
        style="display: none; text-align: center;">
            <img src="../dataset/img/lc_wordvectors_scatter_plot_GROUNDTRUTH.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:66"  src="https://plot.ly/embed.js" async></script>
</div>

## Using the model

#### Analogies
Training the model allows some operations like the famous "king – man + woman = queen“:

Here we are providing the System Programming course **(33)** and its instructor **(253)**, and subtracting another course, Computer Architecture **(32)**

33:  http://cs.illinois.edu/courses/profile/CS241-120158

253: http://cs.illinois.edu/directory/profile/angrave

32:  http://cs.illinois.edu/courses/profile/CS233-120158

and expecting its instructor: http://cs.illinois.edu/directory/profile/zilles **(251)**

In [29]:
# hand is to palm as foot is to ____ (it's sole if you're wondering)
# HAND : PALM : : FOOT : ____   
hand = "33"
palm = "253"
foot = "32"

most_similar_list = word2vec_lc.most_similar(positive=[hand, palm], negative=[foot], topn=1)
sole = most_similar_list[0][0]

print(listcostraint_urlmap[hand].replace("http://cs.illinois.edu/courses/profile/", ""), "is to", 
      listcostraint_urlmap[palm].replace("http://cs.illinois.edu/directory/profile/", ""), "as",
      listcostraint_urlmap[foot].replace("http://cs.illinois.edu/courses/profile/", ""), "is to",
      listcostraint_urlmap[sole].replace("http://cs.illinois.edu/directory/profile/", "")
)

print("")
print(hand, listcostraint_urlmap[hand])
print(palm, listcostraint_urlmap[palm])
print(foot, listcostraint_urlmap[foot])
print(sole, listcostraint_urlmap[sole])

http://www.cs.princeton.edu/events/25360 is to http://www.cs.princeton.edu/~ret as http://www.cs.princeton.edu/news/crouthamel-osborne-and-sanchez-win-special-performance-recognition-award is to http://www.cs.princeton.edu/~dhlarkin

33 http://www.cs.princeton.edu/events/25360
253 http://www.cs.princeton.edu/~ret
32 http://www.cs.princeton.edu/news/crouthamel-osborne-and-sanchez-win-special-performance-recognition-award
219 http://www.cs.princeton.edu/~dhlarkin


#### Doesn't match
Word that doesn't go with the others:

Artificial Intelligence research lab:
- 128 http://cs.illinois.edu/research/artificial-intelligence

Its researchers:
- 304 http://cs.illinois.edu/directory/profile/mrebl
- 271 http://cs.illinois.edu/directory/profile/daf
- 305 http://cs.illinois.edu/directory/profile/juliahmr
- 363 http://cs.illinois.edu/directory/profile/dhoiem

Another guy:
- 361 http://cs.illinois.edu/directory/profile/wgropp


In [30]:
word2vec_lc.doesnt_match("304 271 305 361 363".split())

'305'

---

## Evaluation
Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric.

See the [scikit-learn documentaion](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation "ti") for futher information

### Metrics:

- **Homogeneity**: each cluster contains only members of a single class


- **Completeness**: all members of a given class are assigned to the same cluster


- **Adjusted Rand index**: Given the knowledge of the *ground truth* class assignments and our clustering algorithm assignments of the same samples, the adjusted Rand index is a function that measures the similarity of the two assignments, ignoring permutations and with chance normalization


- **V-measure**: The V-measure is actually equivalent to the mutual information (NMI) discussed above normalized by the sum of the label entropies


- **Mutual Information based scores**: Given the knowledge of the ground truth class assignments and our clustering algorithm assignments of the same samples, the Mutual Information is a function that measures the agreement of the two assignments, ignoring permutations. Two different normalized versions of this measure are available, Normalized Mutual Information(NMI) and Adjusted Mutual Information(AMI). NMI is often used in the literature while AMI was proposed more recently and is normalized against chance


- **Silhouette**: If the ground truth labels are not known, evaluation must be performed using the model itself. The Silhouette Coefficient is an example of such an evaluation, where a higher Silhouette Coefficient score relates to a model with better defined clusters. The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters. The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.

In [31]:
metrics_df = pd.DataFrame([
        [
            # dbscan nocostraint
            metrics.homogeneity_score(ground_truth_nc, dbscan_labels_nc),
            metrics.completeness_score(ground_truth_nc, dbscan_labels_nc),
            metrics.v_measure_score(ground_truth_nc, dbscan_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, dbscan_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, dbscan_labels_nc),
            metrics.silhouette_score(wordvecs_nc, dbscan_labels_nc, metric='euclidean')
        ],
        [
            # hdbscan nocostraint
            metrics.homogeneity_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.completeness_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.v_measure_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.silhouette_score(wordvecs_nc, hdbscan_labels_nc, metric='euclidean')
        ],
        [
            # kmeans nocostraint
            metrics.homogeneity_score(ground_truth_nc, kmeans_labels_nc),
            metrics.completeness_score(ground_truth_nc, kmeans_labels_nc),
            metrics.v_measure_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, kmeans_labels_nc),
            metrics.silhouette_score(wordvecs_nc, kmeans_labels_nc, metric='euclidean')
        ],
        [
            # dbscan listcostraint
            metrics.homogeneity_score(ground_truth_lc, dbscan_labels_lc),
            metrics.completeness_score(ground_truth_lc, dbscan_labels_lc),
            metrics.v_measure_score(ground_truth_lc, dbscan_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, dbscan_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, dbscan_labels_lc),
            metrics.silhouette_score(wordvecs_lc, dbscan_labels_lc, metric='euclidean')
        ],
        [
            # hdbscan listcostraint
            metrics.homogeneity_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.completeness_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.v_measure_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.silhouette_score(wordvecs_lc, hdbscan_labels_lc, metric='euclidean')
        ],
        [
            # kmeans listcostraint
            metrics.homogeneity_score(ground_truth_lc, kmeans_labels_lc),
            metrics.completeness_score(ground_truth_lc, kmeans_labels_lc),
            metrics.v_measure_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, kmeans_labels_lc),
            metrics.silhouette_score(wordvecs_lc, kmeans_labels_lc, metric='euclidean')
        ]],
        index=[
            "NoCostraint - DBSCAN", 
            "NoCostraint - HDBSCAN", 
            "NoCostraint - K-MEANS", 
            "ListCostraint - DBSCAN", 
            "ListCostraint - HDBSCAN", 
            "ListCostraint - K-MEANS"
        ],
        columns=[
            "Homogeneity", 
            "Completeness", 
            "V-Measure core", 
            "Adjusted Rand index", 
            "Mutual Information",
            "Silhouette"
        ])

metrics_df

,Homogeneity,Completeness,V-Measure core,Adjusted Rand index,Mutual Information,Silhouette
NoCostraint - DBSCAN,0.045375,0.196823,0.073748,0.025262,0.039282,0.542013
NoCostraint - HDBSCAN,0.304138,0.387332,0.340730,0.251598,0.295011,0.210031
NoCostraint - K-MEANS,0.400327,0.369018,0.384036,0.345269,0.357233,0.536422
ListCostraint - DBSCAN,0.175140,0.444070,0.251206,0.256214,0.165712,0.354208
ListCostraint - HDBSCAN,0.605157,0.588950,0.596943,0.763833,0.579448,0.330923
ListCostraint - K-MEANS,0.690313,0.411884,0.515931,0.426792,0.396498,0.257934
